In [ ]:
#Import Functions
from m6predictFunctions import * 
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

In [ ]:
#set variables
modelFlank=80
numOut_Classes=3

In [ ]:
#test
halfFlank=int(modelFlank/2) 

InpF=h5py.File('Inp_test.h5','r')
TgtF=h5py.File('Tgt_test.h5','r')

X=[]
Y=[]
for k in InpF.keys():
    X.append(InpF[k][:,5000-halfFlank:-5000+halfFlank])
    Y.append(TgtF[k][:])
tstInpBlocks=np.concatenate(X,axis=0)
tstTgtBlocks=np.concatenate(Y,axis=0)

model=load_model('model_weights_BEST.h5',
                 custom_objects={'tf':tf,'categorical_crossentropy_2d_3ctgs':categorical_crossentropy_2d_3ctgs,
                                'categorical_crossentropy_2d_2ctgs':categorical_crossentropy_2d_2ctgs})
#calculate top-k and PR-AUC accuracies for SDs and SAs (seperatly)

p=model.predict(tstInpBlocks)
print p.shape,tstInpBlocks.shape,tstTgtBlocks.shape

Tks=[]
PRs=[]
for c in range(1,numOut_Classes):
    Fullc=tstTgtBlocks[:,:,c].flatten()
    S=[i for i in range(len(Fullc)) if Fullc[i] == 1]
    pS=np.argsort(p[:,:,c].flatten())[-len(S):]
    pS=pd.Series(pS)
    nCrct=pS[pS.isin(S)].size
    Tk=nCrct/float(len(S))
    Tks.append(Tk)
    
    PR=average_precision_score(Fullc,p[:,:,c].flatten())
    PRs.append(PR)
    if numOut_Classes < 3:
        Tks.append(0.0)
        PRs.append(0.0)
writeAccs=str(Tks[0])+','+str(Tks[1])+','+str(PRs[0])+','+str(PRs[1])
open('Tks_PRs.txt', 'w').write(writeAccs)

In [ ]:
# graph Precision Recall of m6A sites
precision, recall, _ = precision_recall_curve(tstTgtBlocks[:,:,1].flatten(),p[:,:,1].flatten())


plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b')
plt.plot([0,1],[PRs[0],PRs[0]])

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('m6A Sites')

plt.savefig('m6A_Sites.png')

In [ ]:
# graph Precision Recall of comparision site (As or RACs, etc.)
if numOut_Classes > 2:
    plt.figure()

    precision, recall, _ = precision_recall_curve(tstTgtBlocks[:,:,2].flatten(),p[:,:,2].flatten())

    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b')
    plt.plot([0,1],[PRs[1],PRs[1]])

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('third Class Sites')

    plt.savefig('thirdClass.png')